In [3]:
from pymatgen.core.surface import SlabGenerator
from pymatgen import Structure 
from pymatgen.core.surface import generate_all_slabs
import functools 
import os
import pandas as pd

In [54]:
def generate_one_hkl_slab(structure, hkl, thicknesses, vacuums, lll_reduce=True, center_slab=True, make_dirs=False in_unit_planes=False, primitive=True, max_normal_search=None, reorient_lattice=True): 
    """
    generates all unique slabs for a specified hkl with min slab and vacuum thicknesses
    Args: 
        structure (str): filename of structure file 
        hkl (tuple): miller index 
        thicknesses (list): min size in angstroms of the slab 
        vacuums (list): min size in angstroms of vacuum 
        lll_reduce (bool): if true it keeps the slab angles reasonable-ish
        center_slab (bool): if true slab is centred to the middle of unit cell with equal layers of vacuum above and below
        other args are pymatgen defaults 
    Returns: 
        POSCAR_hkl_slab_vac_index.vasp 
    """
    
    struc = Structure.from_file(structure)
    struc.add_oxidation_state_by_guess()
    
    provisional_slabs = []
    
    for thickness in thicknesses: 
        for vacuum in vacuums: 
            slabgen = SlabGenerator(struc, hkl, thickness, vacuum,lll_reduce=lll_reduce, center_slab=center_slab)
            slabs = slabgen.get_slabs()
            for i, slab in enumerate(slabs): 
                if (slab.is_polar() == False) and (slab.is_symmetric() == True): 
                    provisional_slabs.append({'hkl': ''.join(map(str, slab.miller_index)), 
                                              'slab_thickness': thickness, 
                                              'vacuum_thickness': vacuum, 
                                              'slab_index': i, 
                                              'slab': slab})
                    
    unique_slabs_list = []
    unique_slabs_list_of_dicts = []
    repeat_slabs = []
    
    for slab in provisional_slabs: 
        if slab['slab'] not in unique_slabs_list: 
            unique_slabs_list.append(slab['slab'])
            unique_slabs_list_of_dicts.append(slab)
        else:
            repeat_slabs.append('{}_{}_{}_{}'.format(slab['hkl'], slab['slab_thickness'], slab['vacuum_thickness'], slab['slab_index']))
    
    
    if make_dirs is True: 
        pass 
        #tuki pridem do problema ker nism zih kje tocno je cwd v tem primeru 

    for slab in unique_slabs_list_of_dicts: 
        slab['slab'].to(fmt='poscar',filename='POSCAR_{}_{}_{}_{}.vasp'.format(slab['hkl'], slab['slab_thickness'], slab['vacuum_thickness'], slab['slab_index']))
    
    
    if len(provisional_slabs) != len(unique_slabs_list): 
        print('warning: not all combinations slab/vac thickness were generated because of repeat structures. make sure to manually check the missing slabs.')
        print('the repeat slabs are: ' + ', '.join(map(str, repeat_slabs)))


In [10]:

def generate_all_unique_slabs(structure, max_index, thicknesses, vacuums, lll_reduce=True, center_slab=True, bonds=None, tol=0.1, ftol=0.1, max_broken_bonds=0,
                              primitive=True, max_normal_search=None, symmetrize=False, repair=False, include_reconstructions=False, in_unit_planes=False): 
    """ 
    generates all unique slabs with specified max hkl, min slab and vacuum thicknesses; including all combinations for multiple zero-dipole symmetric terminations for the same hkl
    Args: 
        structure: filename of structure file; oxidation states added by guess 
        hkl (tuple): miller index 
        thicknesses (list): min size in angstroms of the slab 
        vacuums (list): min size in angstroms of vacuum 
        lll_reduce (bool): if true it keeps the slab angles reasonable-ish
        center_slab (bool): slab is centred to the middle of unit cell with equal layers of vacuum above and below
        other params are pymatgen defaults  
    Returns: 
        POSCAR_hkl_slab_vac_index.vasp 
    
    """  
    
    struc = Structure.from_file(structure)
    struc.add_oxidation_state_by_guess()
    
    provisional_slabs = []

    for vacuum in vacuums: 
        for thickness in thicknesses:         
            all_slabs = generate_all_slabs(struc, max_index=max_index, min_slab_size=thickness, min_vacuum_size=vacuum, lll_reduce=lll_reduce, center_slab=center_slab)
            for i, slab in enumerate(all_slabs): 
                if (slab.is_polar() == False) and (slab.is_symmetric() == True): 
                    provisional_slabs.append({'hkl': ''.join(map(str, slab.miller_index)), 
                                              'slab_thickness': thickness, 
                                              'vacuum_thickness': vacuum, 
                                              'slab_index': i, 
                                              'slab': slab})
    
    unique_slabs_list = []
    unique_slabs_list_of_dicts = []
    repeat_slabs = []
    
    for slab in provisional_slabs: 
        if slab['slab'] not in unique_slabs_list: 
            unique_slabs_list.append(slab['slab'])
            unique_slabs_list_of_dicts.append(slab)
        else:
            repeat_slabs.append('{}_{}_{}_{}'.format(slab['hkl'], slab['slab_thickness'], slab['vacuum_thickness'], slab['slab_index']))
    

    for slab in unique_slabs_list_of_dicts: 
        slab['slab'].to(fmt='poscar',filename='POSCAR_{}_{}_{}_{}.vasp'.format(slab['hkl'], slab['slab_thickness'], slab['vacuum_thickness'], slab['slab_index']))
    
    if len(provisional_slabs) != len(unique_slabs_list): 
        print('warning: not all combinations of hkl or slab/vac thickness were generated because of repeat structures. make sure to manually check the missing slabs.')
        print('the repeat slabs are: ' + ', '.join(map(str, repeat_slabs)))



In [ ]:
def generate_conv_dirs_one_hkl()

In [ ]:
def generate_conv_dirs(struc, max_index, thicknesses, vacuums, lll_reduce=True, center_slab=True): 
    #ugotovit kako nardis potcars (najlazi pass list zazeljenih in pol nekak to s pymatgenom? dat skupi/), kako nardis incars 

In [11]:
generate_all_unique_slabs('./CONTCAR_conventional', 1, [20,30,40], [20])

the repeat slabs are: 001_40_20_15


In [28]:
filtered_slabs = {}
slabs_we_already_have = []

for vacuum in vacuums: 
    for thickness in thicknesses: 
        filtered_slabs['dict_{}_{}'.format(thickness, vacuum)] = {}

for key, val in acceptable_slabs.items():
    for key2, slab in val.items():
        if slab not in slabs_we_already_have:
            slabs_we_already_have.append(slab)
            filtered_slabs[key][key2] = slab

print(len(filtered_slabs['dict_40_40']))

10


In [ ]:
hkl = ''.join(map(str, slab.miller_index))
slab.to(fmt='poscar',filename='POSCAR_{}_{}_{}_{}.vasp'.format(hkl, thickness, vac, i))

In [7]:
vacuums = [20,30,40]
thicknesses = [20,30,40]

for vacuum in vacuums: 
    for thickness in thicknesses:         
        #all_slabs = generate_all_slabs(structure, max_index=2, min_slab_size=thickness, min_vacuum_size=vacuum, lll_reduce=True, center_slab=True)
        slabgen = SlabGenerator(structure, (1,0,1), min_slab_size=thickness, min_vacuum_size=vacuum, center_slab=True)
        slabs = slabgen.get_slabs()
        for i, slab in enumerate(slabs):
            if (slab.is_polar() == False) and (slab.is_symmetric() == True): 
                hkl = ''.join(map(str, slab.miller_index))
                slab.to(fmt='poscar',filename='POSCAR_{}_{}_{}.vasp'.format(hkl, thickness, vacuum))

In [7]:
slab = slabs[4]
slab.to(fmt='poscar', filename='POSCAR_YTOS_hkl_index.vasp')

In [14]:
slabgen = SlabGenerator(structure,(1,0,0),20,30,lll_reduce=True)
slabs = slabgen.get_slabs()
slab = slabs[12]
slab.to(fmt='poscar', filename='POSCAR.vasp')

In [35]:
slabgen = SlabGenerator(structure,(2,1,2), min_slab_size=40, min_vacuum_size=40)
slabs = slabgen.get_slabs()
for i, slab in enumerate(slabs):
    print(i, slab.is_polar(), slab.is_symmetric())
slab = slabs[2]
slab.to(fmt='poscar', filename='POSCAR_212_40_40_corr.vasp')

0 True False
1 True False
2 False True


In [ ]:
all_slabs = generate_all_slabs(structure, max_index=2, min_slab_size=10, min_vacuum_size=30, lll_reduce=True, center_slab=True)